# Llama-3.2-1B 效能基準測試 (vLLM for Portfolio)

本筆記本使用 **Llama-3.2-1B-Instruct** 進行測試。
選擇此模型的原因：
1. **權限驗證**：它同樣是 Meta 的 Gated Repo，需要 Hugging Face Token 與授權，能展現您處理模型存取的專業能力。
2. **穩定性**：1B 模型體積極小，保證能在 Colab T4 免費版順暢執行，不會發生記憶體崩潰。
3. **高 TPS**：在 T4 上能跑出非常亮眼的推論速度數據。

In [ ]:
# 1. 安裝套件
!pip install vllm pandas huggingface_hub

In [ ]:
# 2. 登入 Hugging Face (請使用 Classic Token)
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 3. 初始化 vLLM (使用 1B 模型)
from vllm import LLM, SamplingParams
import time

# 使用 Llama-3.2-1B，這在 T4 上非常穩定
llm = LLM(model="meta-llama/Llama-3.2-1B-Instruct", 
          dtype="half", 
          gpu_memory_utilization=0.6, 
          max_model_len=2048)

print("✅ vLLM Engine initialized for Llama-3.2-1B!")

In [ ]:
# 4. 執行基準測試
prompts = [
    "The future of autonomous agents is",
    "Explain the concept of backpropagation in one paragraph.",
    "Write a short story about a robot learning to cook.",
    "How does LangGraph help developers build complex agents?"
] * 10  # 40 個請求以測試穩定性

sampling_params = SamplingParams(temperature=0.7, max_tokens=128)

print(f"🚀 正在處理 {len(prompts)} 個請求...")
start_time = time.time()
outputs = llm.generate(prompts, sampling_params)
end_time = time.time()

total_duration = end_time - start_time
total_tokens = sum([len(output.outputs[0].token_ids) for output in outputs])
throughput = total_tokens / total_duration

print(f"\n=== Benchmark Results ===")
print(f"Total Tokens: {total_tokens}")
print(f"Total Time: {total_duration:.2f} s")
print(f"Throughput (TPS): {throughput:.2f} tokens/s")